<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import os
import pandas as pd

In [2]:
import sys
sys.path.append('../../connectTCR')

In [3]:
from connectTCR.utils import import_gliph_output, normalize, get_dynamic_classes, get_trb_groups
from connectTCR.mapping import map_signal_noise, map_shape_cluster, map_overlap
from connectTCR.connection import flux
from connectTCR.plotting import set_figure_params, barplot
from connectTCR.pipeline import plot_connection

In [4]:
gliphdir = '../data/gliph'
dataset = 'blood-signal-csf-signal-tumor'
dataset = 'blood-csf-tumor'

In [5]:
outdir = '../output/{}/'.format(dataset)
if not os.path.isdir(outdir):
    os.system('mkdir -p '+outdir)


In [6]:
df = import_gliph_output(gliphdir, dataset, suffix='_general')
# Rename samples
names = {'blood':'PB', 'csf':'CSF', 'PBT103-1':'Pre', 'PBT120':'Post'}
df['Sample'] = df['Sample'].map(names)

In [7]:
# map signal vs noise
name = {'CSF':'CSF', 'PB':'PBMC'}
signaldir = '../data/tcr_dynamics/'
sample2signal = {comp:pd.read_csv(signaldir+'{c}(Signal).csv'.format(c=name[comp]), index_col=0).reset_index() \
    for comp in name }
df = map_signal_noise(df, sample2signal, 'Sample', 'TcRb', 'index')

In [8]:
# map shape cluster
tcrdir = '../data/tcr_dynamics/'
csfshape = get_dynamic_classes(tcrdir, 'SignalCSFKmeans', 'CSFShapeKmeansMap')
bloodshape = get_dynamic_classes(tcrdir, 'SignalPBMCKmeans', 'PBMCShapeKmeansMap')
sample2shapemap = {'CSF:Signal':csfshape, 'PB:Signal':bloodshape}
df = map_shape_cluster(df, sample2shapemap, 'Sample:Signal/Noise', 'TcRb')
df['Sample:ShapeCluster'] = df['Sample:Signal/Noise:ShapeCluster'].str.replace( \
    ':Signal', '', regex=True)

In [11]:
df1 = get_trb_groups(df, 'CASSELFGSEDIGIEDTF', 
               cols=['index', 'pattern', 'type', 'TcRb', 'TcRa', 'Sample', 'Freq', 'Sample:ShapeCluster'],
               save=True,
               savedir=outdir,
              )

In [12]:
df2 = get_trb_groups(df, 'CSARLAGDYNEQFF', 
               cols=['index', 'pattern', 'type', 'TcRb', 'TcRa', 'Sample', 'Freq', 'Sample:ShapeCluster'],
               save=True,
               savedir=outdir,
              )

In [13]:
df1.pattern.unique()

array(['single'], dtype=object)

In [14]:
df2.pattern.unique()

array(['RLAG%YNE', 'R%AGDYNE', '%LAGDYNE', 'RLAGD%NE', 'RLAGDY%E'],
      dtype=object)